In [ ]:
!nvidia-smi

Wed Jul  7 01:37:25 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')
path = '/content/gdrive/MyDrive/data'
%cd {path}
!ls

Mounted at /content/gdrive
/content/gdrive/MyDrive/data
'Bản sao của eng.txt'			        NNVLP
 char					        OpenNMT-py
 Co_Huong				        OpenNMT-tf
 corpus.21a7f66d40bcf46cc5fba018aba04c7a.data   pred3.txt
 data_err.txt				        pred.txt
 data_e.txt				        rnn_0_epoch.net
 data_pro.txt				        rnn_1_epoch.net
 deep-spell-checkr			        train
 eng1.txt				        train_vntc1.txt
 engt.txt				        VNTC
 eng.txt


In [ ]:
!git clone -b legacy https://github.com/OpenNMT/OpenNMT-py


fatal: destination path 'OpenNMT-py' already exists and is not an empty directory.


In [ ]:
!pip install OpenNMT-py

     |████████████████████████████████| 215kB 7.5MB/s 
     |████████████████████████████████| 102kB 13.0MB/s 
     |████████████████████████████████| 81kB 11.3MB/s 
     |████████████████████████████████| 81kB 12.2MB/s 
     |████████████████████████████████| 14.3MB 26.4MB/s 
     |████████████████████████████████| 61kB 9.9MB/s 
     |████████████████████████████████| 748.8MB 23kB/s 
     |████████████████████████████████| 645kB 58.3MB/s 
     |████████████████████████████████| 1.2MB 69.5MB/s 
ERROR: torchvision 0.10.0+cu102 has requirement torch==1.9.0, but you'll have torch 1.6.0 which is incompatible.
  Found existing installation: Flask 1.1.4
    Uninstalling Flask-1.1.4:
      Successfully uninstalled Flask-1.1.4
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1
  Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
  Found existing inst

In [ ]:
!python OpenNMT-py/preprocess.py -train_src train/train_src2.txt -train_tgt train/train_tgt2.txt -valid_src train/val_src2.txt -valid_tgt train/val_tgt2.txt -save_data Co_Huong/demo

In [ ]:
!nvidia-smi


Sun Jun 27 17:11:37 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

TRAIN MODEL WITH TRANSFORMER


In [ ]:
!python OpenNMT-py/train.py -data train/eng_mark/demo1 -save_model train/eng_mark/model/model -layers 6 -rnn_size 512 -word_vec_size 512 -transformer_ff 2048 -heads 8 -encoder_type transformer -decoder_type transformer -position_encoding -train_steps 150000 -max_generator_batches 2 -dropout 0.1 -batch_size 4096 -batch_type tokens -normalization tokens -accum_count 2 -optim adam -adam_beta2 0.998 -decay_method noam -warmup_steps 8000 -learning_rate 2 -max_grad_norm 0 -param_init 0 -param_init_glorot -label_smoothing 0.1 -valid_steps 1000 -save_checkpoint_steps 1000 -world_size 1 -gpu_rank 0 

In [ ]:
!python OpenNMT-py/translate.py -model train/eng_mark/model/model_step_150000.pt -src Co_Huong/test_src2.txt -output pred.txt -replace_unk -verbose -beam_size 4

In [ ]:
!python OpenNMT-py/translate.py -model train/eng_mark/model/model_step_150000.pt -src train/test_src3.txt -output pred3.txt -replace_unk -verbose -beam_size 2

In [ ]:
!python OpenNMT-py/translate.py -model train/eng_mark/model/model_step_150000.pt -src VNTC/test_src.txt -output VNTC/pred.txt -replace_unk -verbose -beam_size 4

In [ ]:
!perl  OpenNMT-py/tools/multi-bleu.perl Co_Huong/test_tgt2.txt < Co_Huong/test_src2.txt

BLEU = 86.70, 94.2/89.1/84.4/79.8 (BP=1.000, ratio=1.000, hyp_len=9333, ref_len=9329)


In [ ]:
!perl  OpenNMT-py/tools/multi-bleu.perl Co_Huong/test_tgt2.txt < pred.txt

BLEU = 87.13, 95.0/90.5/86.3/82.3 (BP=0.986, ratio=0.986, hyp_len=9196, ref_len=9329)


In [ ]:
!perl  OpenNMT-py/tools/multi-bleu.perl train/test_tgt2.txt < train/test_src2.txt

BLEU = 63.10, 82.8/69.1/57.6/48.1 (BP=1.000, ratio=1.000, hyp_len=251158, ref_len=251176)


In [ ]:
!perl  OpenNMT-py/tools/multi-bleu.perl train/test_tgt2.txt < pred.txt

BLEU = 85.97, 95.3/91.2/87.5/84.0 (BP=0.962, ratio=0.963, hyp_len=149836, ref_len=155661)


In [ ]:
!perl  OpenNMT-py/tools/multi-bleu.perl train/test_tgt2.txt < train/test_src3.txt

BLEU = 64.96, 83.8/70.7/59.7/50.4 (BP=1.000, ratio=1.000, hyp_len=251145, ref_len=251176)


In [ ]:
!perl  OpenNMT-py/tools/multi-bleu.perl train/test_tgt2.txt < pred3.txt

BLEU = 87.10, 95.8/92.1/88.8/85.7 (BP=0.962, ratio=0.963, hyp_len=149890, ref_len=155661)


In [ ]:
!perl  OpenNMT-py/tools/multi-bleu.perl VNTC/test_tgt.txt < VNTC/test_src.txt

BLEU = 63.91, 83.2/69.8/58.6/49.2 (BP=1.000, ratio=1.000, hyp_len=4947898, ref_len=4949719)


In [ ]:
!perl  OpenNMT-py/tools/multi-bleu.perl VNTC/test_tgt.txt < VNTC/pred.txt

BLEU = 83.65, 94.2/89.3/84.8/80.7 (BP=0.960, ratio=0.961, hyp_len=4756177, ref_len=4949719)


In [ ]:
!perl  OpenNMT-py/tools/multi-bleu.perl train/te_gg.txt < train/ggdocs.txt

BLEU = 0.31, 95.1/90.9/87.0/83.2 (BP=0.004, ratio=0.150, hyp_len=1769, ref_len=11766)


In [ ]:
with open('pred3.txt', 'r', encoding='utf-8') as f:
  pred = f.readlines()
print (len(pred))

6030


In [ ]:
with open('train/test_tgt2.txt', 'r', encoding='utf-8') as f:
  test_tgt = f.readlines()
print (len(test_tgt))

10000


In [ ]:
with open('train/test_src3.txt', 'r', encoding='utf-8') as f:
  test_src = f.readlines()
print (len(test_src))

10000


In [ ]:
def wer(r, h):
    """
    Calculation of WER with Levenshtein distance.

    Works only for iterables up to 254 elements (uint8).
    O(nm) time ans space complexity.

    Parameters
    ----------
    r : list
    h : list

    Returns
    -------
    int

    Examples
    --------
    >>> wer("who is there".split(), "is there".split())
    1
    >>> wer("who is there".split(), "".split())
    3
    >>> wer("".split(), "who is there".split())
    3
    """
    # initialisation
    import numpy
    d = numpy.zeros((len(r)+1)*(len(h)+1), dtype=numpy.uint8)
    d = d.reshape((len(r)+1, len(h)+1))
    for i in range(len(r)+1):
        for j in range(len(h)+1):
            if i == 0:
                d[0][j] = j
            elif j == 0:
                d[i][0] = i

    # computation
    for i in range(1, len(r)+1):
        for j in range(1, len(h)+1):
            if r[i-1] == h[j-1]:
                d[i][j] = d[i-1][j-1]
            else:
                substitution = d[i-1][j-1] + 1
                insertion    = d[i][j-1] + 1
                deletion     = d[i-1][j] + 1
                d[i][j] = min(substitution, insertion, deletion)

    return d[len(r)][len(h)]



In [ ]:
from tqdm import tqdm
num = 0
for i in tqdm(range(len(pred))):
  temp = wer(pred[i].split(), test_tgt[i].split())
  num += temp
print(num)

100%|██████████| 6030/6030 [00:43<00:00, 138.17it/s]


In [ ]:
from tqdm import tqdm
num = 0
for i in tqdm(range(len(pred))):
  temp = wer(test_src[i].split(), test_tgt[i].split())
  num += temp
print(num)

100%|██████████| 6030/6030 [00:47<00:00, 126.68it/s]

25230


In [ ]:
def num_right_sen(tgt, pred):
  with open(tgt, 'r', encoding='utf-8') as f:
    tgt1 = f.readlines()
  with open(pred, 'r', encoding='utf-8') as f:
    pre = f.readlines()
  right_sen = 0
  if (len(tgt1) > len(pre)):
    sen_num =len(pre)
  else:
    sen_num = len(tgt1)
  for i in range(sen_num):
    tgt1[i] = tgt1[i].rstrip(' \n')
    #tgtl = tgt1[i].split(' ')
    pre[i] = pre[i].rstrip(' \n')
    #prel = pre[i].split(' ')
    if(i< 10):
      print(tgt1[i])
      print(pre[i])
    if(tgt1[i] == pre[i]):
      right_sen += 1
  acc = float(right_sen*100) / sen_num
  print("Số câu được sửa chính xác là: %d trên %d câu được sửa "%(right_sen,sen_num) )
  print("Độ chính xác theo số câu: " + str(acc) +"%")

In [ ]:
def num_right_word(src,tgt, pred):
  with open(tgt, 'r', encoding='utf-8') as f:
    tgt1 = f.readlines()
  with open(pred, 'r', encoding='utf-8') as f:
    pre = f.readlines()
  with open(src, 'r', encoding='utf-8') as f:
    src1 = f.readlines()
  right_sen = 0
  if (len(tgt1) > len(pre)):
    sen_num =len(pre)
  else:
    sen_num = len(tgt1)
  sai = 0
  sua_dung = 0
  sua_sai = 0
  lam_sai = 0
  sen = 0
  for i in range(sen_num):
    tgt1[i] = tgt1[i].rstrip(' \n')
    tgtl = tgt1[i].split(' ')
    pre[i] = pre[i].rstrip(' \n')
    prel = pre[i].split(' ')
    src1[i] = src1[i].rstrip(' \n')
    srcl = src1[i].split(' ')
    if(len(tgtl) == len(prel) and len(prel) == len(srcl) and i < 100):
      sen += 1
      for j in range(len(tgtl)):
        if(tgtl[j] != srcl[j]):
          sai += 1
          if(tgtl[j] == prel[j]):
            sua_dung += 1
          else:
            sua_sai += 1
        if(tgtl[j] == srcl[j] and tgtl[j] != prel[j]):
          lam_sai +=  1

  print("Số từ được sửa chính xác là: %d trên %d từ sai "%(sua_dung,sai))
  print("Số từ đã sửa nhưng sai là: %d trên %d từ sai "%(sua_sai,sai))
  print("Số từ bị sửa đúng thành sai là: %d từ "%(lam_sai))
  print("Số câu check %d trên %d câu"%(sen,sen_num))

In [ ]:
num_right_word("train/test_src3.txt", "train/test_tgt2.txt", "pred3.txt")

Số từ được sửa chính xác là: 193 trên 238 từ sai 
Số từ đã sửa nhưng sai là: 45 trên 238 từ sai 
Số từ bị sửa đúng thành sai là: 10 từ 
Số câu check 91 trên 6030 câu


In [ ]:
num_right_word("train/src.txt", "train/te_gg.txt", "train/ggdocs.txt")

Số từ được sửa chính xác là: 159 trên 237 từ sai 
Số từ đã sửa nhưng sai là: 78 trên 237 từ sai 
Số từ bị sửa đúng thành sai là: 5 từ 
Số câu check 91 trên 91 câu


In [ ]:
num_right_word("VNTC/test_src.txt", "VNTC/test_tgt.txt", "VNTC/pred.txt")

Số từ được sửa chính xác là: 493834 trên 690720 từ sai 
Số từ đã sửa nhưng sai là: 196886 trên 690720 từ sai 
Số từ bị sửa đúng thành sai là: 41564 từ 
Số câu check 186038 trên 200000 câu


In [ ]:
num_right_sen("train/test_tgt2.txt", "pred.txt")

"Giai điệu, ENG＿7 đường vĩ đại và khó khăn của chúng ta"
"Giai điệu, ENG＿7 đường vĩ đại và khó khăn của chúng ta"
"Trái đất đã cống hiến cho ENG＿7 người tài năng của nó,"
"Trái đất đã cống hiến cho ENG＿7 người tài năng của nó,"
"Và mọi người đã cống hiến tài năng của mình cho Trái đất!"
"Và mọi người đã cống hiến tài năng của mình cho Trái đất!"
Chúng ENG＿67 không giấu mặt khỏi ngọn lửa,
Chúng ENG＿67 không giấu mặt kdỏi ngọn lửa,
Khi kim loại được sinh ra trong lò
Khi kim loại được sinh ra trong lò
Bàn tay mệt mỏi của người thợ rèn được tiết chế với hy vọng bởi giai điệu
Bàn tay mệt mỏi của người thợ rnè được tiết chế với hy vọng bởi giai điệu
Nó sấm sét với quy mô của các nhà máy,
Nó sấm sét với quy mô của các nhà máy,
Và các đoàn tàu mang nó trên đường ENG＿2007
Và các ENG＿7 tàu mang nó trên đường EGN＿2007
Tại nhà của chúng ENG＿67, nó nói chuyện với chúng ENG＿67 -
Tại nhà của chúng ENG＿67, nó nói chuyện với chúng ENG＿67 -
Giai điệu công nhân của Kuzbass
Giai điệu hạt nhân của Kuzbass
